<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_%EC%A0%9C3%EC%9C%A0%ED%98%95_AI%EB%AC%B8%EC%A0%9C_%EC%B0%90%EB%A7%89_%EB%81%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# 데이터 생성 (시험 환경과 동일하게 복사해서 사용하세요)
data = pd.DataFrame({
    'id': range(1, 21),
    'temperature': [20.5, 22.1, 24.3, 26.5, 28.1, 21.0, 23.5, 25.8, 27.2, 29.5,
                    20.1, 21.8, 24.0, 26.2, 28.5, 19.8, 22.5, 25.0, 27.8, 29.1],
    'wind_speed': [3.5, 4.2, 2.1, 1.8, 5.5, 3.2, 4.0, 2.5, 1.5, 6.0,
                   3.8, 4.5, 2.3, 1.6, 5.2, 3.0, 4.3, 2.8, 1.2, 5.8],
    'humidity': [45, 50, 55, 60, 40, 48, 52, 58, 62, 38,
                 46, 51, 56, 59, 41, 44, 53, 57, 63, 39],
    'ozone': [30, 35, 40, 45, 50, 28, 38, 42, 48, 55,
              32, 36, 41, 46, 52, 29, 37, 43, 49, 53]
})

In [26]:
#문제1
from scipy import stats
c = 0.95
l = len(data['ozone'])
s = data['ozone'].std()
m = data['ozone'].mean()

res = stats.t.interval(confidence = c, df = l-1, loc =  m, scale = s/l**0.5)
print(res[0]) #37.58028805204542

37.58028805204542


In [28]:
#문제2
from scipy import stats
c = 0.99
l = len(data['ozone'])
s = data['ozone'].std()
m = data['ozone'].mean()

res = stats.t.interval(confidence = c, df = l-1, loc = m, scale = s/(l**0.5))
print(res[1])# 46.73947233338222

46.73947233338222


In [31]:
from statsmodels.formula.api import ols
model = ols('ozone~temperature+wind_speed+humidity',data=data).fit()
new_data = pd.DataFrame({
    'temperature' : [25], 'wind_speed':[3.0], 'humidity': [50]
})
pred = model.get_prediction(new_data)
res = pred.summary_frame(alpha=0.05)
print(res.loc[0, 'mean_ci_lower']) #40.97113284820103

40.97113284820103


In [32]:
model = ols('ozone~temperature+wind_speed+humidity', data=data).fit()
new_data = pd.DataFrame({
    'temperature' : [25], 'wind_speed' : [3.0], 'humidity' : [50]
})
pred = model.get_prediction(new_data)
res = pred.summary_frame(0.05)
res.loc[0, 'mean_ci_upper']

np.float64(42.87462906526505)

In [43]:
model = ols('ozone~temperature+wind_speed+humidity', data=data).fit()
new_data = pd.DataFrame({
    'temperature' : [25], 'wind_speed' : [3.0], 'humidity' : [50]
})
pred = model.get_prediction(new_data)
res = pred.summary_frame(alpha = 0.1)
print(res)
print(res.loc[0, 'mean_ci_upper'] - res.loc[0, 'mean_ci_lower']) #1.5676563356140605

        mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  41.922881  0.448958      41.139053      42.706709     39.259288   

   obs_ci_upper  
0     44.586473  
1.5676563356140605


In [42]:
#문제 6. 데이터 중 humidity가 50 이상인 데이터만 추출하여, 해당 데이터의 wind_speed 변수에 대한
#95% 신뢰구간의 상한값을 구하시오. (t-분포 사용)
new_data = data[data['humidity'] >= 50]
print(new_data)

from scipy import stats
c = 0.95
l = len(new_data['wind_speed'])
m = new_data['wind_speed'].mean()
s = new_data['wind_speed'].std()
interval = stats.t.interval(confidence = c, df = l-1, loc = m, scale = s/(l**0.5))
print(interval[1]) #3.4997776342689964

    id  temperature  wind_speed  humidity  ozone
1    2         22.1         4.2        50     35
2    3         24.3         2.1        55     40
3    4         26.5         1.8        60     45
6    7         23.5         4.0        52     38
7    8         25.8         2.5        58     42
8    9         27.2         1.5        62     48
11  12         21.8         4.5        51     36
12  13         24.0         2.3        56     41
13  14         26.2         1.6        59     46
16  17         22.5         4.3        53     37
17  18         25.0         2.8        57     43
18  19         27.8         1.2        63     49
3.4997776342689964


In [48]:
#문제 7. 종속변수를 temperature, 독립변수를 humidity로 하는 단순 선형 회귀 모델을 만들고,
#humidity가 45일 때의 99% 예측구간(Prediction Interval)의 하한값을 구하시오.
from statsmodels.formula.api import ols
model = ols('temperature~humidity', data=data).fit()
new_data = pd.DataFrame({'humidity' : [45]})

pred = model.get_prediction(new_data)
res = pred.summary_frame(alpha=0.01)
print(res.loc[0, 'obs_ci_lower']) #14.97582969446902


14.97582969446902


In [57]:
#문제 8. temperature 변수에 대해 95% 신뢰구간을 구했을 때, (상한값 - 평균) 의 값을 구하시오.
#(즉, 오차한계(Margin of Error)를 구하시오)
from scipy import stats
c = 0.95
l = len(data['temperature'])
m = data['temperature'].mean()
s = data['temperature'].std()

res = stats.t.interval(confidence = c, df = l-1, loc = m, scale = s/(l**0.5))
print(res[1], m) #26.143532212047177 24.665
print(res[1] - m) #1.4785322120471776

26.143532212047177 24.665
1.4785322120471776


In [59]:
# 문제 9. 3번 문제의 모델(다중회귀)에서 새로운 데이터(temperature: 28, wind_speed: 2.0, humidity: 45)
#에 대한 95% 예측구간의 폭과 신뢰구간의 폭의 차이를 구하시오. (예측구간 폭 - 신뢰구간 폭)
from statsmodels.formula.api import ols
model = ols('ozone~temperature+wind_speed+humidity',data=data).fit()

new_data = pd.DataFrame({
    'temperature' : [28], 'wind_speed' : [2.0], 'humidity' : [45]
})
pred = model.get_prediction(new_data)
res = pred.summary_frame(alpha=0.05)
print(res)

obs = res.loc[0, 'obs_ci_upper'] - res.loc[0, 'obs_ci_lower']
mean = res.loc[0, 'mean_ci_upper'] - res.loc[0, 'mean_ci_lower']
print(obs-mean) #2.594859472616392

        mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  48.406022  1.430868      45.372717      51.439328     44.075288   

   obs_ci_upper  
0     52.736757  
2.594859472616392


In [63]:
# 문제 10. wind_speed 변수에 대해 표본평균의 90% 신뢰구간을 구하고, 구간의 **중심값(Center)**이
# 표본평균과 일치하는지 확인하여 표본평균 값을 제출하시오.
from scipy import stats
l = len(data['wind_speed'])
m = data['wind_speed'].mean()
s = data['wind_speed'].std()
res = stats.t.interval(confidence = 0.9, df = l-1, loc = m, scale = s/(l**0.5))
print(res)

middle = (res[0] + res[1])/2

if m == middle:
  print('일치')
else:
  print('안 일치')
#일치

(np.float64(2.866182343944457), np.float64(4.013817656055543))
일치


※ 모든 정답은 소수점 넷째 자리에서 반올림하여 셋째 자리까지 기재하시오.

문제 1. ozone 변수에 대해 표본평균의 95% 신뢰구간(Confidence Interval)을 구하고, 그 중 **하한값(Lower Bound)**을 구하시오. (단, scipy.stats.t.interval을 사용할 것)

문제 2. ozone 변수에 대해 표본평균의 99% 신뢰구간을 구하고, 그 중 **상한값(Upper Bound)**을 구하시오.

문제 3. 종속변수를 ozone, 독립변수를 temperature, wind_speed, humidity로 하는 다중 선형 회귀 모델(OLS)을 수립하시오. temperature가 25, wind_speed가 3.0, humidity가 50일 때의 데이터를 기반으로 예측했을 때, 95% 신뢰구간(Confidence Interval)의 하한값을 구하시오.

문제 4. 위 3번 문제와 동일한 모델 및 새로운 데이터를 사용하여, 95% 예측구간(Prediction Interval)의 상한값을 구하시오.

문제 5. 위 3번 문제와 동일한 모델 및 새로운 데이터를 사용하여, 유의수준 0.1(90% 신뢰수준)에서의 **신뢰구간(Confidence Interval)의 폭(Width)**을 구하시오. (폭 = 상한값 - 하한값)

문제 6. 데이터 중 humidity가 50 이상인 데이터만 추출하여, 해당 데이터의 wind_speed 변수에 대한 95% 신뢰구간의 상한값을 구하시오. (t-분포 사용)

문제 7. 종속변수를 temperature, 독립변수를 humidity로 하는 단순 선형 회귀 모델을 만들고, humidity가 45일 때의 99% 예측구간(Prediction Interval)의 하한값을 구하시오.

문제 8. temperature 변수에 대해 95% 신뢰구간을 구했을 때, (상한값 - 평균) 의 값을 구하시오. (즉, 오차한계(Margin of Error)를 구하시오)

문제 9. 3번 문제의 모델(다중회귀)에서 새로운 데이터(temperature: 28, wind_speed: 2.0, humidity: 45)에 대한 95% 예측구간의 폭과 신뢰구간의 폭의 차이를 구하시오. (예측구간 폭 - 신뢰구간 폭)

문제 10. wind_speed 변수에 대해 표본평균의 90% 신뢰구간을 구하고, 구간의 **중심값(Center)**이 표본평균과 일치하는지 확인하여 표본평균 값을 제출하시오.